In [1]:
# Edgar new data Jan 2024

import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import pickle

import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from models import *
from utils import *

seed = 2468
np.random.seed(seed)
torch.manual_seed(seed)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.cuda.get_device_name()

'NVIDIA GeForce RTX 3090'

In [2]:
'''
The imaging is still at 30hz.
The ephys is binarized in 1ms bins.
The speed has corresponding triggers in imaging and ephys.
I.e. for every entry in the speed file, there is a corresponding imaging frame or ephys bin at the given index.
There are files for baseline and for stimulation. In each case, the imaging ROIs and the ephys units are matched.
I.e. index 1 in baseline imaging is the same cell as index 1 is stim imaging.

Please see if this format works for you. Let me know if you want me to downsample all of them to the same frame rate (30Hz) instead if that is easier for you. If this runs ok, then I can run another 1-2 mice through which should put us at 3-4 animals. I will work on getting the combinatorial paradigm in place as well, but it is a little bit complicated right now.
'''
area_ID_ephys = np.load('./data/biological/EB095/area_ID_ephys.npy')        # (106,) 8 LS, 70 ACC
dff_run = np.load('./data/biological/EB095/dff_run.npy')                    # (294, 33897) GCaMP when running
dff_stim = np.load('./data/biological/EB095/dff_stim.npy')                  # (294, 111810) GCaMP when stimulated 
spks_run = np.load('./data/biological/EB095/spks_run.npy')                  # (106, 1188000)
spks_stim = np.load('./data/biological/EB095/spks_stim.npy')                # (106, 3799862)
speed_run = np.load('./data/biological/EB095/speed_run.npy')                # (56302,) max 13

speed_triggers_ephys_run = np.load('./data/biological/EB095/speed_triggers_ephys_run.npy')  # (56322)
speed_triggers_img_run = np.load('./data/biological/EB095/speed_triggers_img_run.npy')      # (56322)
stim_ID = np.load('./data/biological/EB095/stim_ID.npy')                    # (300,), R S or F
stim_triggers_ephys_stim = np.load('./data/biological/EB095/stim_triggers_ephys_stim.npy')  # (300,)
stim_triggers_img_stim = np.load('./data/biological/EB095/stim_triggers_img_stim.npy')      # (300,)

In [6]:
stim_triggers_ephys_stim

array([3829245, 3849633, 3860027, 3870436, 3880863, 3891287, 3901697,
       3912108, 3922512, 3932920, 3943333, 3953747, 3964160, 3974565,
       3984968, 3995390, 4005802, 4016214, 4026632, 4037045, 4047435,
       4057836, 4068243, 4078648, 4089053, 4099468, 4109885, 4120301,
       4130714, 4141127, 4151513, 4161929, 4172346, 4182765, 4193179,
       4203567, 4213959, 4224371, 4234793, 4245209, 4255613, 4266009,
       4276404, 4286822, 4297219, 4307622, 4318006, 4328407, 4338818,
       4349225, 4359649, 4370070, 4380462, 4390885, 4401275, 4411667,
       4422076, 4432498, 4442902, 4453311, 4463735, 4474157, 4484580,
       4494997, 4505413, 4515822, 4526241, 4536661, 4547068, 4557461,
       4567875, 4578297, 4588716, 4599131, 4609551, 4619958, 4630374,
       4640779, 4651190, 4661598, 4672017, 4682421, 4692814, 4703215,
       4713620, 4724022, 4734440, 4744855, 4755269, 4775679, 4786087,
       4796502, 4806909, 4817325, 4827732, 4838144, 4848549, 4858966,
       4869377, 4879

Naive model fitting:
Ignore running / stationary, fit model to both data.

In [80]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
lm.fit(stim_triggers_img_stim[:,None],stim_triggers_ephys_stim[:,None])
freq = lm.coef_[0][0] # * 1000 Hz
intercept = lm.intercept_ # succeeding time

In [114]:
n_bins = int(spks_stim.shape[1]/freq)

spks_stim_30Hz = np.zeros((spks_stim.shape[0], n_bins)) # (106, 114141)
# stim_triggers_ephys_stim_30Hz = np.array((stim_triggers_ephys_stim - intercept) / freq, dtype=np.int64)
for n in range(n_bins):
    spks_stim_30Hz[:,n] = np.sum(spks_stim[:, int(n*freq): int((n+1)*freq)], axis=1)